In [1]:
import pandas as pd
import umap
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import PredefinedSplit, GridSearchCV, cross_val_score, cross_val_predict, LeaveOneOut, KFold
from sklearn.svm import SVC
from hypopt import GridSearch

from sklearn.preprocessing import OneHotEncoder

from umap import UMAP
import skdim
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import seaborn as sns

from sklearn.neighbors import NearestNeighbors

from sklearn.decomposition import PCA, FastICA
# import psyplot.project as psy
from sklearn.svm import SVR

/home/jc225751/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/jc225751/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


ImportError: Numba needs NumPy 1.21 or less

In [ ]:
model_path = "/neurospin/dico/jchavas/Runs/70_self-supervised_two-regions/Output/CINGULATE_right/16-18-46_176"
ukb_emb_file = f"{model_path}/ukb_random_embeddings/full_embeddings.csv"

In [ ]:
ukb_emb = pd.read_csv(ukb_emb_file, index_col=0)
ukb_emb

# UKBioBank

In [ ]:
# norm = ukb_emb.apply(np.linalg.norm, axis=1)
# ukb_emb = ukb_emb.div(norm, axis=0)

In [ ]:
ukb_emb

In [ ]:
p = plt.hist(ukb_emb.values.ravel(), bins=np.arange(-0.1,0.4,0.01), density=True)

In [ ]:
Z = ukb_emb.values.ravel()
N = len(Z)
X2 = np.sort(Z)
F2 = np.array(range(N))/float(N)

plt.plot(X2, F2)

In [ ]:
X2[int(N*0.59)]

In [ ]:
len(ukb_emb)

In [ ]:
ukb_positive = (ukb_emb>0).sum(axis=0)/len(ukb_emb)
ukb_positive = ukb_positive.sort_values(ascending=False)

In [ ]:
ukb_positive

In [ ]:
plt.plot(ukb_positive)

In [ ]:
p = plt.hist(ukb_emb.loc[:,"dim83"], bins=100, density=True)

In [ ]:
p = plt.hist(ukb_emb.loc[:,"dim65"], bins=100, density=True)

# UMAP

In [ ]:
def compute_umap(df, n_neighbors=15, n_components=2):
    # apply umap
    reducer = umap.UMAP(n_neighbors=n_neighbors, n_components=n_components)
    visual_embs = reducer.fit_transform(df)
    return visual_embs, reducer

In [ ]:
visual_ukb_emb, reducer = compute_umap(ukb_emb, n_neighbors=15)

In [ ]:
ukb_emb.head()

In [ ]:
reduced_ukb_emb, reducer_red = compute_umap(ukb_emb, n_neighbors=15, n_components=20)

In [ ]:
plt.scatter(reduced_ukb_emb[:,0], reduced_ukb_emb[:,1], c='black', s=1, alpha=0.6)

In [ ]:
reduced_ukb_emb.shape

In [ ]:
umap_ukb = visual_ukb_emb

In [ ]:
plt.scatter(visual_ukb_emb[:,0], visual_ukb_emb[:,1], c='black', s=1, alpha=0.6)

In [ ]:
h = plt.hist(reduced_ukb_emb[:,0], bins=100)

In [ ]:
h = plt.hist(reduced_ukb_emb[:,0], bins=100)

In [ ]:
plt.scatter(reduced_ukb_emb[:,14], reduced_ukb_emb[:,15], c='black', s=1, alpha=0.6)

In [ ]:
plt.scatter(reduced_ukb_emb[:,4], reduced_ukb_emb[:,5], c='black', s=1, alpha=0.6)

# UKB latent space analysis

In [ ]:
global_id = skdim.id.MOM().fit(ukb_emb, n_neighbors=5000)

In [ ]:
global_id.dimension_

In [ ]:
umap_id = skdim.id.MOM().fit(reduced_ukb_emb, n_neighbors=1000)
umap_id.dimension_

In [ ]:
h = plt.hist(umap_id.dimension_pw_, bins=100)

In [ ]:
# lid_mom = skdim.id.lPCA().fit_pw(ukb_emb, n_neighbors=50, n_jobs=1)

In [ ]:
# lid_mom.dimension_

In [ ]:

h = plt.hist(global_id.dimension_pw_, bins=100)

In [ ]:
plt.figure(figsize=(12,8))
q_min = np.min(global_id.dimension_pw_)
q_05 = np.quantile(global_id.dimension_pw_, 0.05) 
q_95 = np.quantile(global_id.dimension_pw_, 0.95)
q_max = np.max(global_id.dimension_pw_)
cmap, norm = mcolors.from_levels_and_colors([q_min, q_05, q_95, q_max], ['red', 'white', 'blue'])
alpha = []
plt.scatter(umap_ukb[:,0], umap_ukb[:,1], s=20, c=global_id.dimension_pw_, alpha=0.6, cmap=cmap, norm=norm)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
q_min = np.min(umap_id.dimension_pw_)
q_05 = np.quantile(umap_id.dimension_pw_, 0.05) 
q_95 = np.quantile(umap_id.dimension_pw_, 0.95)
q_max = np.max(umap_id.dimension_pw_)
cmap, norm = mcolors.from_levels_and_colors([q_min, q_05, q_95, q_max], ['red', 'white', 'blue'])
alpha = []
plt.scatter(umap_ukb[:,0], umap_ukb[:,1], s=20, c=umap_id.dimension_pw_, alpha=0.6, cmap=cmap, norm=norm)
plt.show()

In [ ]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm='brute').fit(ukb_emb)
distances, indices = nbrs.kneighbors(ukb_emb)

In [ ]:
h = plt.hist(distances[:,1], bins=100)

In [ ]:
heatmap, xedges, yedges = np.histogram2d(distances[:,1], global_id.dimension_pw_, bins=(50,50))
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap.T, origin='lower', extent=extent, aspect='auto', cmap=cm.jet)
# plt.figure(figsize=(8,100))
plt.show()

# UMAP of only lower dimensions

In [ ]:
ukb_red = ukb_emb.copy(deep=True)
ukb_red.insert(0, "id", global_id.dimension_pw_)

In [ ]:
ukb_red.insert(1, "umap_0", umap_ukb[:,0])
ukb_red.insert(2, "umap_1", umap_ukb[:,1])
ukb_red.insert(3, "distance", distances[:,1])
ukb_red

In [ ]:
ukb_red = ukb_red[ukb_red.id < np.quantile(global_id.dimension_pw_, 0.05) ]
# ukb_red = ukb_red[ukb_red.umap_0 > 6]

In [ ]:
ukb_red.head()

In [ ]:
plt.scatter(ukb_red.umap_0, ukb_red.umap_1, s=20)

In [ ]:
ukb_umap_red, reducer_red = compute_umap(ukb_red[4:], n_neighbors=15)

In [ ]:
red_id = skdim.id.MOM().fit(ukb_red[4:], n_neighbors=500)
red_id.dimension_

In [ ]:
h = plt.hist(red_id.dimension_pw_, bins=100)

In [ ]:
plt.figure(figsize=(12,8))
q_min = np.min(red_id.dimension_pw_)
q_05 = np.quantile(red_id.dimension_pw_, 0.05) 
q_95 = np.quantile(red_id.dimension_pw_, 0.95)
q_max = np.max(red_id.dimension_pw_)
cmap, norm = mcolors.from_levels_and_colors([q_min, 3.5, q_95, q_max], ['red', 'green', 'blue'])
alpha = []
plt.scatter(ukb_umap_red[:,0], ukb_umap_red[:,1], s=20, c=red_id.dimension_pw_, alpha=0.6, cmap=cmap, norm=norm)
plt.show()

In [ ]:
heatmap, xedges, yedges = np.histogram2d(ukb_red.distance, ukb_red.id, bins=(50,50))
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap.T, origin='lower', extent=extent, aspect='auto', cmap=cm.jet)
# plt.figure(figsize=(8,100))
plt.show()

In [ ]:
ukb_red

In [ ]:
ukb_red2 = ukb_red[ukb_red.umap_0 > 6]
ukb_red2

In [ ]:
my_model = PCA(n_components=20)
my_model.fit_transform(ukb_emb)
explained_all = my_model.explained_variance_ratio_.cumsum()

In [ ]:
my_model = PCA(n_components=20)
my_model.fit_transform(ukb_red2[4:])
explained = my_model.explained_variance_ratio_.cumsum()
plt.plot(explained, 'r')
plt.plot(explained_all)

In [ ]:
reduced_ukb_emb.shape

In [ ]:
my_model = PCA(n_components=20)
on_axes = my_model.fit_transform(reduced_ukb_emb)
explained = my_model.explained_variance_ratio_.cumsum()
plt.plot(explained, 'r')
plt.plot(explained_all)

In [ ]:
explained

In [ ]:
on_axes.shape

In [ ]:
plt.scatter(on_axes[:,0], on_axes[:,1], s=1)

In [ ]:
plt.scatter(on_axes[:,0], on_axes[:,2], s=1)

In [ ]:
plt.scatter(on_axes[:,2], on_axes[:,3], s=1)

In [ ]:
heatmap, xedges, yedges = np.histogram2d(on_axes[:,0], on_axes[:,1], bins=(50,50))
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap.T, origin='lower', extent=extent, aspect='auto', cmap=cm.jet)
# plt.figure(figsize=(8,100))
plt.colorbar()
plt.show()

In [ ]:
heatmap, xedges, yedges = np.histogram2d(on_axes[:,2], on_axes[:,3], bins=(50,50))
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap.T, origin='lower', extent=extent, aspect='auto', cmap=cm.jet)
# plt.figure(figsize=(8,100))
plt.show()

In [ ]:
from sklearn.metrics import r2_score
from sklearn.cross_decomposition import PLSRegression

X = ukb_emb
umap_embedding = reduced_ukb_emb

#MNIST variation explained by UMAP1
UMAP_matrix = pd.DataFrame(umap_embedding)
pls = PLSRegression(n_components = 1)
pls.fit(UMAP_matrix, X)
y_pred = pls.predict(UMAP_matrix)
print(r2_score(X, y_pred, multioutput = 'variance_weighted'))
#0.07335034485651613

#Here the same but more explicitly via the R^2 equation above
print(1 - np.sum((np.array(X) - np.array(y_pred))**2) / np.sum((X - \
np.mean(X, axis = 0))**2))
#0.07335034485652026


#Here the same but explicitly via the R^2 equation above
print(1 - np.sum((np.array(X) - np.array(y_pred))**2) / np.sum((X - \
np.mean(X, axis = 0))**2))
#0.07265918428991347

In [ ]:
X = reduced_ukb_emb

explainable = np.zeros((X.shape[1],1))
for i in range(X.shape[1]):
    umap_embedding = reduced_ukb_emb[:,i]

    #MNIST variation explained by UMAP1
    UMAP_matrix = pd.DataFrame(umap_embedding)
    pls = PLSRegression(n_components = 1)
    pls.fit(UMAP_matrix, X)
    y_pred = pls.predict(UMAP_matrix)
    print(r2_score(X, y_pred, multioutput = 'variance_weighted'))
    #0.07335034485651613

    #Here the same but more explicitly via the R^2 equation above
    # print(1 - np.sum((np.array(X) - np.array(y_pred))**2) / np.sum((X - \
    # np.mean(X, axis = 0))**2))
    #0.07335034485652026


    #Here the same but explicitly via the R^2 equation above
    explainable[i] = 1 - np.sum((np.array(X) - np.array(y_pred))**2) / np.sum((X - \
    np.mean(X, axis = 0))**2)
    #0.07265918428991347

In [ ]:
plt.plot(explainable.cumsum())